In [4]:
import os
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/alexamgreat/loan-pay-back.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "alexamgreat"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "3a89fd7489fff804a2a2ef260ce8c6773ad8561f"

In [5]:
import os
%pwd

'c:\\Users\\ASUS\\Desktop\\loan-pay-back\\research'

In [6]:
os.chdir("../")
%pwd

'c:\\Users\\ASUS\\Desktop\\loan-pay-back'

In [7]:
from dataclasses import dataclass
from pathlib import Path    

@dataclass
class ModelEvaluationConfig:
    root_dir:Path
    test_data_dir:Path
    model_dir:Path
    metrics_file_name:Path
    all_params:dict
    target_column:str
    mlflow_uri:str

In [8]:
from src.loan_payment_prediction.constants import*
from src.loan_payment_prediction.utils.common import read_yaml, create_directories, save_json   


In [9]:
class ModelEvaluationConfigManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.SGDRegressor
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_dir=config.test_data_dir,
            model_dir=config.model_dir,
            metrics_file_name=config.metrics_file_name,
            all_params=self.params,
            target_column= schema.name,
            mlflow_uri="https://dagshub.com/alexamgreat/loan-pay-back.mlflow"
        )

        return model_evaluation_config

In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow   
import mlflow.sklearn
import numpy as np
import joblib 


In [29]:
from sklearn import metrics


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, predicted) -> dict:
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)

        return {
            "rmse": rmse,
            "mae": mae,
            "r2_score": r2
        }

    def log_into_mlflow(self):
        #close any existing mlflow runs
        if mlflow.active_run() is not None:
            mlflow.end_run()
        # Load data and model
        test_data = pd.read_csv(self.config.test_data_dir)
        model = joblib.load(self.config.model_dir)

        # SAFE pandas usage (NO list, NO Box)
        test_x = test_data.drop(columns=self.config.target_column, axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predictions = model.predict(test_x)
            metrics = self.eval_metrics(test_y, predictions)

            rmse  = metrics["rmse"]
            mae   = metrics["mae"]
            r2    = metrics["r2_score"]
        # Save metrics locally  
        metrics_path = Path(self.config.root_dir) / self.config.metrics_file_name
        metrics_path.parent.mkdir(parents=True, exist_ok=True)

        save_json(path=metrics_path, data=metrics)  

        # Log params correctly
        mlflow.log_params(self.config.all_params)

# Log metrics to MLflow
        mlflow.log_metric("rmse", metrics["rmse"])
        mlflow.log_metric("mae", metrics["mae"])
        mlflow.log_metric("r2_score", metrics["r2_score"])

        
        
        # Log model
        if tracking_uri_store != "file":
            mlflow.sklearn.log_model(
                model,
                "model",
                registered_model_name="LoanPayBackModel"
            )
        else:
            mlflow.sklearn.log_model(model, "model")
        print("Model evaluation metrics logged into MLflow successfully.")  

In [30]:
try:
    config_manager = ModelEvaluationConfigManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()  
    
except Exception as e:
    raise e

[2025-12-25 11:55:28,110]: INFO: YAML file config\config.yaml loaded successfully.
[2025-12-25 11:55:28,114]: INFO: YAML file params.yaml loaded successfully.
[2025-12-25 11:55:28,119]: INFO: YAML file schema.yaml loaded successfully.
[2025-12-25 11:55:28,122]: INFO: Directory created at: artifacts
[2025-12-25 11:55:28,124]: INFO: Directory created at: artifacts/model_evaluation
🏃 View run entertaining-jay-968 at: https://dagshub.com/alexamgreat/loan-pay-back.mlflow/#/experiments/0/runs/adb8a50f04f74031b63f99cb50c564aa
🧪 View experiment at: https://dagshub.com/alexamgreat/loan-pay-back.mlflow/#/experiments/0
🏃 View run caring-colt-683 at: https://dagshub.com/alexamgreat/loan-pay-back.mlflow/#/experiments/0/runs/5da8e56dadf94898a2b5d834d24cd119
🧪 View experiment at: https://dagshub.com/alexamgreat/loan-pay-back.mlflow/#/experiments/0
[2025-12-25 11:55:31,797]: INFO: JSON file saved at: artifacts\model_evaluation\metrics.json


2025/12/25 11:55:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/25 11:55:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'LoanPayBackModel'.
2025/12/25 11:55:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LoanPayBackModel, version 1
Created version '1' of model 'LoanPayBackModel'.


Model evaluation metrics logged into MLflow successfully.
